# Обогащение имеющегося датасета новыми данными

# Вступительная база

## Импорт необходимых библиотек

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from io import StringIO

## Функции, испольщуемые далее

In [7]:
def merge_custom(df1, df2, month = False, day = False, hour = False, Q = False):
    on_list = ['year']
    if hour:
        month, day = True, True
        on_list.append('hour')
    if day:
        month = True
        on_list.append('day')
    if month:
        on_list.append('month')
    if Q:
        on_list.append('Q')
    return df1.merge(df2, on=on_list, how='left')

# Считаем датасет и получим новые солбцы с временем

In [110]:
df = pd.read_csv('data/test.csv')
df['date'] = pd.to_datetime(df['date'])

In [112]:
def split_date(df, date_row, day=False, hour = False, Q = False):
    df1 = df.copy()
    df1['year'] = df1[date_row].dt.year
    df1['month'] = df1[date_row].dt.month
    if hour:
        day = True
        df1['hour'] = df1[date_row].dt.hour
    if day:
        df1['day'] = df1[date_row].dt.day
    if Q:
        df1['Q'] = df1[date_row].dt.quarter
    return df1

In [114]:
df = split_date(df, 'date', hour=True, Q=True)

# Добавим в датасет праздничные и выходные дни

In [117]:
def generate_url(year):
    return f"https://www.work-day.co.uk/data_excel?from={year}-01-01&to={year}-12-31&step=1&cols=52,0,1;52,0,3;52,0,11;52,0,12;52,0,2;&country_code_adding=england"

def fetch_table_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', id='data_table')
        if table:
            table_html = StringIO(str(table))
            return pd.read_html(table_html)[0]
        else:
            print(f"Таблица не найдена на странице: {url}")
            return None
    else:
        print(f"Не удалось получить данные: {response.status_code}")
        return None

def get_data_for_years(start_year, end_year):
    all_data = pd.DataFrame()
    for year in range(start_year, end_year + 1):
        url = generate_url(year)
        year_data = fetch_table_data(url)
        if year_data is not None:
            all_data = pd.concat([all_data, year_data], ignore_index=True)
    return all_data

def rename_columns(data):
    """Переименовывает столбцы в DataFrame."""
    rename_map = {
        'Export to Excel $ 1.99': 'date',
        '× England, working days': 'woring_days',
        '× England, public holidays': 'public_holidays',
        '× England, work hours (h)': 'work_hours',
        '× England, wages (£)': 'wages',
        '× England, weekend days': 'weekend_days'
    }
    return data.rename(columns=rename_map)

In [119]:
data = get_data_for_years(1979, 2020)
data = rename_columns(data)
data = data.drop(['work_hours', 'wages'], axis = 1) # useless

In [120]:
def clear_date(df):
    data = df.copy()
    data[['day_of_week', 'day', 'month', 'year']] = data['date'].str.extract(
        r'(?P<day_of_week>\w+),\s(?P<day>\d+)\s(?P<month>\w+),\s(?P<year>\d{4})'
    )
    
    data['day'] = data['day'].astype(int)
    data['year'] = data['year'].astype(int)
    
    day_of_week_map = {
        'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6, 'Sun': 7
    }
    
    month_map = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }
    
    data['day_of_week'] = data['day_of_week'].map(day_of_week_map)
    data['month'] = data['month'].map(month_map)
    
    data = data.drop('date', axis = 1)
    return data
data = clear_date(data)

In [123]:
df = merge_custom(df, data, day=True)

# Добавим статистику по лицензированным таксистам в каждый год

In [126]:
taxi = pd.ExcelFile("data/taxi0101.ods")
taxi.close()
taxi = taxi.parse('TAXI0101a')[['Year', 'London licensed taxis (thousands) [note 1]']]
taxi['London licensed taxis (thousands) [note 1]'] = taxi['London licensed taxis (thousands) [note 1]']*1000
taxi = taxi.rename(columns = {'London licensed taxis (thousands) [note 1]':'licensed taxis', 'Year':'year'})
df = merge_custom(df, taxi)
# df['licensed taxis'].fillna(df['licensed taxis'].median(), inplace=True)

# Добавим статистику по количеству ДТП

In [129]:
# casualty = pd.read_csv('data/dft-road-casualty-statistics-casualty-1979-latest-published-year.csv') #здесь можно найти насколько пострадали участники дтп
collision = pd.read_csv('data/dft-road-casualty-statistics-collision-1979-latest-published-year.csv')
vehicle = pd.read_csv('data/dft-road-casualty-statistics-vehicle-1979-latest-published-year.csv')

In [130]:
#нас интересуют только эти столбцы
collision = collision[['accident_index', 'date', 'local_authority_district', 'time' ]]

In [133]:
# коды районов лондона
london_boroughs = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,57,570]

In [135]:
collision = collision[collision['local_authority_district'].isin(london_boroughs)]
#получили индексы аварий, которые были совершены в лондоне, их дату и время

In [137]:
# нас инетересуют только эти столбцы
vehicle = vehicle[['accident_index', 'vehicle_type']]

In [139]:
# vehicle_type == 1 - велосипед
cycle_collision = collision.merge(vehicle[vehicle['vehicle_type'] == 1], on='accident_index') #количество дтп с велосипедистами
all_collision = collision #все дтп


#группируем все дтп по 24 часа
cycle_collision['datetime'] = pd.to_datetime(cycle_collision['date'] + ' ' + cycle_collision['time'], dayfirst=True)
all_collision['datetime'] = pd.to_datetime(all_collision['date'] + ' ' + all_collision['time'], dayfirst=True)

cycle_collision = cycle_collision['datetime'].to_frame().sort_values('datetime')
all_collision = all_collision['datetime'].to_frame().sort_values('datetime')

cycle_collision = cycle_collision.set_index('datetime').resample('h').size().rolling('24h').sum().to_frame(name="collision_cycle_count").reset_index()
all_collision = all_collision.set_index('datetime').resample('h').size().rolling('24h').sum().to_frame(name="collision_all_count").reset_index()

In [140]:
cycle_collision = cycle_collision.rename(columns={'datetime':'date'})
all_collision = all_collision.rename(columns={'datetime':'date'})

In [143]:
df = df.merge(cycle_collision, on='date', how='left')
df = df.merge(all_collision, on='date', how='left')

In [145]:
#наны только в начале, так что заполним их нулями
df['collision_cycle_count'].fillna(0, inplace=True)
df['collision_all_count'].fillna(0, inplace=True)

# Claimant
ежемесячный процент заявителей в лондоне по месяцам

In [148]:
claimant = pd.read_excel('data/Claimant Count  E12000007 London  People  SA  Percentage (%).xls')
claimant['date'] = pd.to_datetime(claimant['year'])
claimant = split_date(claimant, 'date')
claimant = claimant.drop(columns='date')

df = merge_custom(df, claimant, month=True)

# CPIH на мотоциклы и велосипедисты
ежемесячные данные с января 1988

In [151]:
motorcycles_and_bicycles = pd.read_excel('data/CPIH INDEX 07.1.23 Motorcycles and bicycles 2015=100.xls')
motorcycles_and_bicycles['date'] = pd.to_datetime(motorcycles_and_bicycles['year'])
motorcycles_and_bicycles = split_date(motorcycles_and_bicycles, 'date')
motorcycles_and_bicycles = motorcycles_and_bicycles.drop(columns='date')

df = merge_custom(df, motorcycles_and_bicycles, month=True)

# GDP UK
ежеквартальные данные

In [154]:
gdp = pd.read_csv('data/GDP_UK.csv')
gdp[['year', 'Q']] = gdp['year'].str.split(' ', expand=True)
gdp['year'] = gdp['year'].astype(int)
gdp['Q'] = gdp['Q'].str.replace('Q', '').astype(int)

df = merge_custom(df, gdp, Q=True)

# Инфляция UK
ежемесячные данные с января 1989 года

In [157]:
inflation = pd.read_csv('data/Inflation_UK.csv')
inflation['date'] = pd.to_datetime(inflation['year'])
inflation = split_date(inflation, 'date')
inflation = inflation.drop(columns=['date'])

df = merge_custom(df, inflation, month=True)

# Economic activity rate
ежемесячные данные с апреля 1992

In [160]:
activity_rate = pd.read_excel('data/LFS Economic activity rate London Aged 16-64 All % SA.xls')
activity_rate['date'] = pd.to_datetime(activity_rate['year'])
activity_rate = split_date(activity_rate, 'date')
activity_rate = activity_rate.drop(columns=['date'])

df = merge_custom(df, activity_rate, month=True)

# Количество работающих 
ежемесячные данные с апреля 1992

In [163]:
in_employment = pd.read_csv('data/LFS In employment London All Thousands SA.csv')

in_employment['date'] = pd.to_datetime(in_employment['year'])
in_employment = split_date(in_employment, 'date')
in_employment = in_employment.drop(columns=['date'])

df = merge_custom(df, in_employment, month=True)

# Заполненность Темзы
ежедневные данные с января 1989

In [166]:
#Данные с января 89 года
reservoir_level = pd.read_excel('data/london-reservoir-levels.xlsx', sheet_name='Daily')
reservoir_level = split_date(reservoir_level, 'Date', day=True)
reservoir_level = reservoir_level.drop(columns=['Lee_level_capacity', 'Date'])
df = merge_custom(df, reservoir_level, day=True)

# Популяция
по кварталам с апреля 1984, по месяцам с апреля 1992

In [169]:
population = pd.read_csv('data/Population aged 16 and over London All Thousands NSA.csv')
population['date'] = pd.to_datetime(population['year'])
population = split_date(population, 'date')
population = population.drop(columns = 'date')

df = merge_custom(df, population, month=True)

# RPI на алкоголь и табак
ежемесячные данные с февраля 1987

In [172]:
#данные с февраля 1987 года
alco_and_tobacco = pd.read_csv('data/RPIPercentage change over 1 month - Alcohol and tobacco.csv')
alco_and_tobacco['date'] = pd.to_datetime(alco_and_tobacco['year'])
alco_and_tobacco = split_date(alco_and_tobacco, 'date')
alco_and_tobacco = alco_and_tobacco.drop(columns = 'date')

df = merge_custom(df, alco_and_tobacco, month=True)

# Процент безработных
ежемесячные данные с апреля 1992

In [175]:
#данные с апреля 1992
unemployment_rate = pd.read_csv('data/unemployment rate London.csv')
unemployment_rate['date'] = pd.to_datetime(unemployment_rate['year'])
unemployment_rate = split_date(unemployment_rate, 'date')
unemployment_rate = unemployment_rate.drop(columns = 'date')

df = merge_custom(df, unemployment_rate, month=True)

# Погода

In [178]:
import meteostat as mt
from datetime import datetime

In [180]:
# посмотрим, какие есть станции в радиусе 50 км от лондона
stations = mt.Stations()
stations = stations.nearby(lat = 51.5085, lon = -0.1257, radius=100 * 1000)
stations = stations.fetch()
stations.head()

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
03779,London Weather Centre,GB,ENG,03779,EGRB,51.5167,-0.1167,5.0,Europe/London,1992-04-01,2010-02-01,1992-04-03,2010-01-31,2006-01-01,2009-01-01,1104.206082
EGLC0,London / Abbey Wood,GB,ENG,<NA>,EGLC,51.5000,0.1167,5.0,Europe/London,1988-01-29,2024-11-07,2007-09-26,2022-04-27,2016-01-01,2022-01-01,16804.070848
03672,Northolt,GB,ENG,03672,EGWU,51.5500,-0.4167,38.0,Europe/London,1973-01-01,2024-11-07,1973-01-05,2022-04-25,2005-01-01,2022-01-01,20652.355896
03772,London Heathrow Airport,GB,ENG,03772,EGLL,51.4833,-0.4500,24.0,Europe/London,1948-12-01,2024-11-07,1948-12-01,2024-10-30,1948-01-01,2022-01-01,22624.394649
03781,Kenley,GB,ENG,03781,<NA>,51.3000,-0.0833,170.0,Europe/London,2018-01-27,2024-11-07,2018-01-28,2022-04-23,2018-01-01,2022-01-01,23369.949135


In [182]:
import warnings

warnings.filterwarnings('ignore')

In [86]:
# посмотрим сколько пропусков на каждой станции
start = datetime(1979, 1, 1)
end = datetime(2020, 12, 31, 23, 59)
for station in stations.index:
    weather_hourly = mt.Hourly(station, start, end)
    weather_hourly = weather_hourly.fetch()
    if weather_hourly.first_valid_index() <= datetime(2000, 1, 1) and weather_hourly.last_valid_index() == datetime(2020, 12, 31, 23):
        print(station, stations.loc[station][0], weather_hourly.shape)
        print(weather_hourly.info())
        print('\n\n')

03779 London Weather Centre (174723, 11)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 174723 entries, 1992-04-01 00:00:00 to 2020-12-31 23:00:00
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   temp    174526 non-null  float64
 1   dwpt    174426 non-null  float64
 2   rhum    174426 non-null  float64
 3   prcp    0 non-null       float64
 4   snow    0 non-null       float64
 5   wdir    142661 non-null  float64
 6   wspd    142669 non-null  float64
 7   wpgt    19851 non-null   float64
 8   pres    173340 non-null  float64
 9   tsun    0 non-null       float64
 10  coco    22875 non-null   float64
dtypes: float64(11)
memory usage: 16.0 MB
None



EGLC0 London / Abbey Wood (197520, 11)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 197520 entries, 1988-01-29 14:00:00 to 2020-12-31 23:00:00
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   temp 

URLError: <urlopen error [Errno 54] Connection reset by peer>

увы, но prcp никто не отмечал и coco на каждой станции мало, не хватит для нормальной обучающей выборки

In [184]:
# возьмем London Heathrow Airport
start = datetime(1979, 1, 1)
end = datetime(2020, 12, 31, 23, 59)
weather_hourly = mt.Hourly('03772', start, end)
weather_hourly = weather_hourly.fetch()

In [186]:
weather_hourly = weather_hourly.reset_index()[['temp', 'dwpt', 'rhum', 'wdir', 'wspd', 'pres', 'time']]
weather_hourly['time'] = pd.to_datetime(weather_hourly['time'])
weather_hourly = split_date(weather_hourly, 'time', hour=True)
weather_hourly = weather_hourly.drop(columns=['time'])
df = merge_custom(df, weather_hourly, hour=True)

In [188]:
#
#
#
# дневные prcp и snow не будем добавлять, я пока еще поищу 
#
#
#
#

# Время рассвета и заката

In [191]:
from astral import LocationInfo
from astral.sun import sun
from astral.location import Location
import datetime
import pandas as pd

In [193]:
dates = pd.date_range(start="1979-01-01", end="2020-12-31", freq="D")

city = LocationInfo("London", "England")
sun_data = []
for date in dates:
    s = sun(city.observer, date=date, tzinfo=city.timezone)
    sun_data.append({
        'date' : date,
        'sunrise' : s['sunrise'],
        'sunset' : s['sunset']
    })
sun_data = pd.DataFrame(sun_data)

sun_data['sunrise'] = sun_data['sunrise'].dt.time
sun_data['sunset'] = sun_data['sunset'].dt.time

In [194]:
sun_data = split_date(sun_data, 'date', day=True)
sun_data.drop(columns='date', inplace=True)
df = merge_custom(df, sun_data, day=True)

In [197]:
df['darkness'] = (~((df['sunrise'] < df['date'].dt.time) & (df['date'].dt.time < df['sunset']))).astype(int)
df.drop(columns=['sunrise', 'sunset'], inplace=True)

# Фазы луны

In [200]:
from astral import moon

In [202]:
moon_data = []
for date in dates:
    m = moon.phase(date=date)
    moon_data.append({
        'date' : date,
        'moon_phase_value' : m,
    })
moon_data = pd.DataFrame(moon_data)

def get_moon_phase(moon_phase_value):
    if 0 <= moon_phase_value < 7:
        return 'New Moon'
    elif 7 <= moon_phase_value < 14:
        return 'First Quarter'
    elif 14 <= moon_phase_value < 21:
        return 'Full Moon'
    elif 21 <= moon_phase_value < 28:
        return 'Last Quarter'

moon_data['moon_phase'] = moon_data['moon_phase_value'].apply(get_moon_phase)

In [204]:
moon_data = split_date(moon_data, 'date', day=True)
moon_data.drop(columns=['date'], inplace=True)
df = merge_custom(df, moon_data, day=True)

# Сохраняем Итог

In [207]:
# df.to_csv('./data/train_udp.csv', index = False)
df.to_csv('./data/test_udp.csv', index = False)
df

,pressure,cloud_cover,evgeniy_gennadievich_beer,sunshine,tarot,min_temp,date,max_temp,snow_depth,mean_temp,...,unemployment_rate,temp,dwpt,rhum,wdir,wspd,pres,darkness,moon_phase_value,moon_phase
0,101860.0,8,0.0,0.0,"('The Devil', 'The World')",4.8,2017-01-01 00:00:00,7.5,0.0,7.5,...,5.9,6.7,5.2,90.0,200.0,18.4,1024.1,1,2.911222,New Moon
1,101860.0,8,0.0,0.0,"('The High Priestess', 'Justice')",4.8,2017-01-01 01:00:00,7.5,0.0,7.5,...,5.9,6.2,5.2,93.0,210.0,13.0,1022.7,1,2.911222,New Moon
2,101860.0,8,0.0,0.0,"('The Empress', 'The Magician')",4.8,2017-01-01 02:00:00,7.5,0.0,7.5,...,5.9,6.0,5.0,93.0,210.0,16.6,1021.9,1,2.911222,New Moon
3,101860.0,8,0.0,0.0,"('The Lovers', 'The Star')",4.8,2017-01-01 03:00:00,7.5,0.0,7.5,...,5.9,5.7,4.8,94.0,200.0,13.0,1020.7,1,2.911222,New Moon
4,101860.0,8,0.0,0.0,"('The Empress', 'The Star')",4.8,2017-01-01 04:00:00,7.5,0.0,7.5,...,5.9,5.6,4.6,93.0,210.0,14.8,1019.6,1,2.911222,New Moon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,100500.0,7,0.0,1.3,"('The Tower', 'The Star')",-3.1,2020-12-31 19:00:00,1.5,0.0,-0.8,...,7.5,0.4,-0.9,91.0,330.0,3.6,1007.6,1,15.277889,Full Moon
35060,100500.0,7,0.1,1.3,"('The Magician', 'The Sun')",-3.1,2020-12-31 20:00:00,1.5,0.0,-0.8,...,7.5,0.7,-0.8,90.0,320.0,1.8,1008.0,1,15.277889,Full Moon
35061,100500.0,7,0.1,1.3,"('The High Priestess', 'The Lovers')",-3.1,2020-12-31 21:00:00,1.5,0.0,-0.8,...,7.5,0.8,-0.7,90.0,360.0,3.6,1008.4,1,15.277889,Full Moon
35062,100500.0,7,0.1,1.3,"('The Hierophant', 'The Hermit')",-3.1,2020-12-31 22:00:00,1.5,0.0,-0.8,...,7.5,1.0,-0.5,90.0,320.0,5.4,1008.6,1,15.277889,Full Moon
